In [107]:
import pandas as pd

stock_price_by_min = pd.read_csv("../data/raw/TSLA_5_year_by_min.csv")
stock_price_by_min_2018_2020 = pd.read_csv("../data/raw/TSLA_2018_to_2020.csv")
tweets = pd.read_csv("../data/raw/elonmusk_raw.csv")

stock_price_by_min = pd.concat(
    [stock_price_by_min, stock_price_by_min_2018_2020]
).drop_duplicates()

print(stock_price_by_min.info())
print(" ------------------------------------------------------ ")
print(tweets.info())

/tmp/ipykernel_143405/3146432059.py:5: DtypeWarning: Columns (11,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv("../data/raw/elonmusk_raw.csv")


<class 'pandas.core.frame.DataFrame'>
Index: 1602029 entries, 0 to 461420
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   ts_event       1602029 non-null  object 
 1   rtype          1602029 non-null  int64  
 2   publisher_id   1602029 non-null  int64  
 3   instrument_id  1602029 non-null  int64  
 4   open           1602029 non-null  float64
 5   high           1602029 non-null  float64
 6   low            1602029 non-null  float64
 7   close          1602029 non-null  float64
 8   volume         1602029 non-null  int64  
 9   symbol         1602029 non-null  object 
dtypes: float64(4), int64(4), object(2)
memory usage: 134.4+ MB
None
 ------------------------------------------------------ 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55099 entries, 0 to 55098
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  ----

In [ ]:
# Lets construct the dataset to predict the stock price in the next X minutes after a
# tweet is posted
import numpy as np

NUM_MINUTES_AHEAD = 5

tweet_wanted_attributes = ["createdAt", "isRetweet", "isReply", "isQuote", "fullText"]
stock_wanted_attributes = ["ts_event", "open", "close"]

# filtering for only original tweets
X = tweets[tweet_wanted_attributes]
X = X[(X["isRetweet"] == False) & (X["isReply"] == False) & (X["isQuote"] == False)]
print(f"Number of original tweets: {len(X)} / {len(tweets)}")

# converting createdAt to datetime
X["createdAt"] = pd.to_datetime(X["createdAt"])

# featurizing day of week and hour of day using sin and cos transformations
X["day_sin"] = X["createdAt"].dt.dayofweek.apply(lambda x: np.sin(2 * np.pi * x / 7))
X["day_cos"] = X["createdAt"].dt.dayofweek.apply(lambda x: np.cos(2 * np.pi * x / 7))
X["hour_sin"] = X["createdAt"].dt.hour.apply(lambda x: np.sin(2 * np.pi * x / 24))
X["hour_cos"] = X["createdAt"].dt.hour.apply(lambda x: np.cos(2 * np.pi * x / 24))

# lets now get the stock prices NUM_MINUTES_AHEAD after each tweet
stock_price_by_min["ts_event"] = pd.to_datetime(stock_price_by_min["ts_event"])
y = stock_price_by_min[stock_wanted_attributes]

# create window start and end times to join on
X["window_start_time"] = X["createdAt"].dt.ceil("min")
X["window_end_time"] = X["window_start_time"] + pd.Timedelta(minutes=NUM_MINUTES_AHEAD)

X = X.sort_values("createdAt").reset_index(drop=True)
y = y.sort_values("ts_event").reset_index(drop=True)

# filter X to only have tweets that have stock price data after them
X = X[y["ts_event"].min() <= X["window_start_time"]]
# filter to weekdays and market hours (data is in UTC)
# this range is a little wider to account for Daylight Savings Time changes
X = X[
    (X["createdAt"].dt.dayofweek < 5)
    & (X["createdAt"].dt.hour >= 13)
    & (X["createdAt"].dt.hour < 21)
]

# join to get stock prices at the start of the window
data = pd.merge_asof(
    X, y, left_on="window_start_time", right_on="ts_event", direction="forward"
)
# join to get stock prices at the end of the window
data = pd.merge_asof(
    data,
    y,
    left_on="window_end_time",
    right_on="ts_event",
    direction="forward",
    suffixes=("_start", "_end"),
)

# 1. Use the sorted y for the rolling average calculation.
y_for_merge = y.set_index("ts_event").copy()

# 2. Calculate the rolling average of the PREVIOUS 5 minutes.
# If this is still constant, the issue is in the 'y' DataFrame itself.
y_for_merge["avg_last_5_prices"] = (
    y_for_merge["close"].rolling(window="5min", closed="left", min_periods=1).mean()
)
# 3. Merge this new feature back into 'data'.
data = pd.merge_asof(
    data,
    y_for_merge["avg_last_5_prices"].reset_index(),
    left_on="window_start_time",
    right_on="ts_event",
    direction="backward",
)

data["avg_last_5_prices"] = (
    10000 * (data["avg_last_5_prices"] - data["open_start"]) / data["open_start"]
)

# calculate the % change and convert to basis points (0.01%) so we dont have to deal
# with small decimals
data["bps_change"] = (
    10000 * (data["close_end"] - data["open_start"]) / data["open_start"]
)

# 4. Drop the extra ts_event column created by the merge_asof
data = data.drop(columns=["ts_event"])

# drop unnecessary columns
data = data.drop(
    columns=[
        "createdAt",
        "isRetweet",
        "isReply",
        "isQuote",
        "ts_event_start",
        "ts_event_end",
        "open_start",
        "open_end",
        "close_start",
        "close_end",
        "window_start_time",
        "window_end_time",
    ]
)
data.dropna(inplace=True)
print(data.info())
data.head()

Number of original tweets: 6857 / 55099
1330
1326
<class 'pandas.core.frame.DataFrame'>
Index: 1326 entries, 0 to 1329
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fullText           1326 non-null   object 
 1   day_sin            1326 non-null   float64
 2   day_cos            1326 non-null   float64
 3   hour_sin           1326 non-null   float64
 4   hour_cos           1326 non-null   float64
 5   avg_last_5_prices  1326 non-null   float64
 6   bps_change         1326 non-null   float64
dtypes: float64(6), object(1)
memory usage: 82.9+ KB
None


,fullText,day_sin,day_cos,hour_sin,hour_cos,avg_last_5_prices,bps_change
0,Oh and uh short burn of the century comin soon...,-0.433884,-0.900969,-0.258819,-0.965926,-19.728035,1.056859
1,Rococo basilisk,0.000000,1.000000,-0.965926,0.258819,1.268231,44.054334
2,Night at the museum. G has mad skillzsz. Me no...,0.974928,-0.222521,-0.965926,-0.258819,8.836035,11.294180
3,@NannySoleil @CatherineUllo We sketched concep...,0.974928,-0.222521,-0.965926,-0.258819,7.653910,19.966722
4,It’s super messed up that a Tesla crash result...,0.000000,1.000000,-0.866025,0.500000,10.984107,-2.402773


In [109]:
# Now we need to vectorize the text. Let's use a pre-trained embedding model
from sentence_transformers import SentenceTransformer

# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# encode the full text to get embeddings - embedding dim is 384
embeddings = model.encode(data["fullText"].tolist(), show_progress_bar=True)
print("Shape of embedding tensor:", embeddings.shape)

Batches: 100%|██████████| 42/42 [00:05<00:00,  7.14it/s]

Shape of embedding tensor: (1326, 384)


In [110]:
# Let's create a dataframe for the embeddings
embedding_cols = [f"emb_{i}" for i in range(embeddings.shape[1])]

# Add as separate columns (concate horizontally matching on same index)
embedding_df = pd.DataFrame(embeddings, columns=embedding_cols, index=data.index)
data = pd.concat([embedding_df, data], axis=1)

# drop unnecessary fullText column since we have it vectorized now
data_unchanged = data.copy()
data = data.drop(columns=["fullText"])

data.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_380,emb_381,emb_382,emb_383,day_sin,day_cos,hour_sin,hour_cos,avg_last_5_prices,bps_change
0,-0.001008,0.052315,-0.009976,-0.014350,0.043964,-0.011720,-0.076216,-0.042937,-0.015438,-0.032219,...,0.016286,-0.039692,0.012335,0.026120,-0.433884,-0.900969,-0.258819,-0.965926,-19.728035,1.056859
1,-0.079920,-0.016484,-0.065066,0.012709,0.020921,-0.029911,-0.025508,0.067164,-0.005198,-0.059973,...,-0.007372,-0.064891,0.070080,0.054620,0.000000,1.000000,-0.965926,0.258819,1.268231,44.054334
2,-0.113694,0.040588,-0.036774,-0.022655,-0.058566,-0.035179,0.050456,-0.056513,-0.025926,-0.040562,...,0.101047,0.040583,-0.122994,-0.003087,0.974928,-0.222521,-0.965926,-0.258819,8.836035,11.294180
3,-0.079638,-0.041545,-0.010374,-0.044825,0.038043,-0.031383,0.033809,0.011510,0.006959,0.023301,...,0.036524,0.021378,-0.023749,0.066412,0.974928,-0.222521,-0.965926,-0.258819,7.653910,19.966722
4,-0.020206,-0.010676,0.045836,0.060031,0.089319,0.014100,-0.034083,0.032532,-0.035500,0.059870,...,0.012382,0.012327,-0.004536,0.081259,0.000000,1.000000,-0.866025,0.500000,10.984107,-2.402773


In [111]:
from sklearn.model_selection import train_test_split

# split it into train, val and test sets
X, y = data[data.columns[:-1]], data["bps_change"]

# temporal split - no shuffling. 70-15-15 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, shuffle=False
)

print(f"Train size: {len(X_train)}, Val size: {len(X_val)}, Test size: {len(X_test)}")

Train size: 928, Val size: 199, Test size: 199


In [112]:
# Lets finally fit a linear regression model for a baseline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def fit_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)

    # Get metrics
    R2_train = model.score(X_train, y_train)
    MSE_train = mean_squared_error(y_train, model.predict(X_train))

    R2_val = model.score(X_val, y_val)
    MSE_val = mean_squared_error(y_val, model.predict(X_val))

    print(f"Train R2: {R2_train}, Train MSE: {MSE_train}")
    print(f"Validation R2: {R2_val}, Validation MSE: {MSE_val}")


fit_model(LinearRegression(), X_train, y_train, X_val, y_val)

Train R2: 0.43261037848957296, Train MSE: 888.5217877001062
Validation R2: -2.5342832819737646, Validation MSE: 2193.427823880905


In [113]:
# Fit regularized linear models - Ridge and Lasso
from sklearn.linear_model import Lasso, Ridge

# default hyperparameters used
print("=== Ridge Regression ===")
fit_model(Ridge(alpha=1.0), X_train, y_train, X_val, y_val)

print("=== Lasso Regression ===")
fit_model(Lasso(alpha=1.0), X_train, y_train, X_val, y_val)

=== Ridge Regression ===
Train R2: 0.2914190912933722, Train MSE: 1109.6247655326713
Validation R2: -0.561743056987362, Validation MSE: 969.2405508128509
=== Lasso Regression ===
Train R2: 0.012799328450232683, Train MSE: 1545.9382270143005
Validation R2: -0.10192047599349885, Validation MSE: 683.8679412246878


In [114]:
# lets try some tree based algorithms - bagged and boosted approach
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

print("=== Random Forest Regressor ===")
model = RandomForestRegressor(n_estimators=100)
fit_model(model, X_train, y_train, X_val, y_val)

print("=== Gradient Boosting Regressor ===")
model = GradientBoostingRegressor(n_estimators=100)
fit_model(model, X_train, y_train, X_val, y_val)

# massively overfits, but gets good training performance, so the model seems expressive
# enough for the task

=== Random Forest Regressor ===
Train R2: 0.8550634929052283, Train MSE: 226.96792381227763
Validation R2: -0.395927940096767, Validation MSE: 866.3332674994483
=== Gradient Boosting Regressor ===
Train R2: 0.7503057027929305, Train MSE: 391.0167104261527
Validation R2: -0.2978706397897539, Validation MSE: 805.4774747776116


In [115]:
# lets try a few regularized models by reducing max depth

print("=== Random Forest Regressor ===")
model = RandomForestRegressor(n_estimators=10, max_depth=15)
fit_model(model, X_train, y_train, X_val, y_val)

=== Random Forest Regressor ===
Train R2: 0.6464045595835951, Train MSE: 553.7240036309278
Validation R2: -0.6145469039012095, Validation MSE: 1002.011389420926


In [116]:
print("=== Gradient Boosting Regressor ===")
model = GradientBoostingRegressor(n_estimators=10, max_depth=15, min_samples_leaf=10)
fit_model(model, X_train, y_train, X_val, y_val)

=== Gradient Boosting Regressor ===
Train R2: 0.4729049964686304, Train MSE: 825.4211516572119
Validation R2: -0.12408173755491458, Validation MSE: 697.6215438204496


In [117]:
y_pred = model.predict(X_test)
squared_errors = (y_test - y_pred) ** 2
overall_mse = mean_squared_error(y_test, y_pred)

error_df = X_test.copy()
error_df["True_Value"] = y_test.values
error_df["Predicted_Value"] = y_pred
error_df["Squared_Error"] = squared_errors.values

# 5. Sort the DataFrame by the Squared_Error column (descending)
sorted_error_df = error_df.sort_values(by="Squared_Error", ascending=False)

# 6. Extract the top 20 rows
top_20_errors = sorted_error_df.head(20)
high_error_indices = top_20_errors.index.tolist()

original_high_error_text = data_unchanged.loc[
    high_error_indices, ["fullText", "bps_change"]
]
print(original_high_error_text)


original_text_df = original_high_error_text
original_text_df.columns = ["Original_Text", "bps_change"]

# Merge the original text with the top_20_errors DataFrame
# We only need the index, True_Value, Predicted_Value, and Squared_Error columns
analysis_df = top_20_errors[["True_Value", "Predicted_Value", "Squared_Error"]].copy()

# Join the original text (which shares the same index)
final_analysis_df = analysis_df.join(original_text_df)

print("\n--- Final Analysis of Top 20 High-Error Samples ---")
# Display the DataFrame, sorting by error descending again for clarity
# print(final_analysis_df.sort_values(by="Squared_Error", ascending=False))
final_analysis_df.to_csv("high_error.csv")

                                               fullText  bps_change
1316                            https://t.co/VnLcmywp6g -234.034833
1268      This is so crazy!\n\n https://t.co/p8B3dXrkAW -179.485418
1269  Increasingly significant consideration https:/... -145.140419
1266  European justice is broken \n https://t.co/Pcf... -166.553596
1267      Thunder Dome vibes \n https://t.co/gYVNl1ufPS -136.544531
1302                            https://t.co/szHlftGMee -113.597394
1169  Unless @DOGE ends the careers of deceitful, po... -128.738769
1170  Your elected representatives have heard you an... -124.640894
1265  It might make sense to increase compensation f...   89.331554
1327                            https://t.co/8thYN2waCI   98.937497
1328                          😉 https://t.co/jR6ycgAYjc  -93.183120
1175                                  Sounds reasonable  -87.433179
1155  Mind-blowing @USGAO report: https://t.co/YaA3o...  -80.030200
1171  You are the media now\n\n https://t.co/kuQ

In [119]:
sorted_error_df = error_df.sort_values(by="Squared_Error", ascending=True)
bottom_50_errors = sorted_error_df.head(50)
low_error_indices = bottom_50_errors.index.tolist()

original_low_error_text = data_unchanged.loc[
    low_error_indices, ["fullText", "bps_change"]
]
print(original_low_error_text)

original_text_df = original_low_error_text
original_text_df.columns = ["Original_Text", "bps_change"]

# Merge the original text with the top_20_errors DataFrame
# We only need the index, True_Value, Predicted_Value, and Squared_Error columns
analysis_df = bottom_50_errors[
    ["True_Value", "Predicted_Value", "Squared_Error"]
].copy()

# Join the original text (which shares the same index)
final_analysis_df = analysis_df.join(original_text_df)

print("\n--- Final Analysis of Bottom 50 Low-Error Samples ---")
# Display the DataFrame, sorting by error descending again for clarity
# print(final_analysis_df.sort_values(by="Squared_Error", ascending=False))
final_analysis_df.to_csv("low_error.csv")

                                               fullText  bps_change
1132  A fully reusable rocket with orbital refilling...  -10.157137
1309  Extremely important difference https://t.co/yu...   -0.718313
1165  Testing streaming on X via Starlink https://t....   -8.155583
1308                      Yup\n https://t.co/B8m7hTmND7   -7.145664
1180  Naturally, I would NEVER be an attention-seeki...    8.963904
1228   Greatest President ever! https://t.co/U8BuPVwSwd    8.590143
1323                          😂 https://t.co/gBcA9SLl1a    1.838506
1150  It was a coordinated sychological operation \n...    4.166419
1234           Check out my bio https://t.co/vAUdrHVDW5    1.504031
1152                                    Defund the ACLU   -6.210479
1273  President @realDonaldTrump is the Commander-in...  -10.187592
1288        So much fake gnus\n https://t.co/BGZQiMhhX0    5.211674
1136  So many life lessons to be learned from speedr...   -2.055076
1189        Testing X Streaming https://t.co/f8L